# How to add cross-thread persistence to your graph

<div class="admonition tip">
    <p class="admonition-title">Prerequisites</p>
    <p>
        This guide assumes familiarity with the following:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    Persistence
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/memory/">
                    Memory
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#chat-models/">
                    Chat Models
                </a>
            </li>             
        </ul>
    </p>
</div>

In the [previous guide](https://langchain-ai.github.io/langgraph/how-tos/persistence/) you learned how to persist graph state across multiple interactions on a single [thread](). LangGraph also allows you to persist data across **multiple threads**. For instance, you can store information about users (their names or preferences) in a shared memory and reuse them in the new conversational threads.

In this guide, we will show how to construct and use a graph that has a shared memory implemented using the [Store](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) interface.

<div class="admonition note">
    <p class="admonition-title">Note</p>
    <p>
    Support for the <code><a href="https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore">Store</a></code> API that is used in this guide was added in LangGraph <code>v0.2.32</code>.
    </p>
    <p>
    Support for <b>index</b> and <b>query</b> arguments of the <code><a href="https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore">Store</a></code> API that is used in this guide was added in LangGraph <code>v0.2.54</code>.
    </p>
</div>

## Setup

First, let's install the required packages and set our API keys

In [1]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")
_set_env("OPENAI_API_KEY")

ANTHROPIC_API_KEY:  ········
OPENAI_API_KEY:  ········


!!! tip "Set up [LangSmith](https://smith.langchain.com) for LangGraph development"

    Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started [here](https://docs.smith.langchain.com)

## Define store

In this example we will create a graph that will be able to retrieve information about a user's preferences. We will do so by defining a `RedisStore` - an object that can store data in Redis and query that data. We will then pass the store object when compiling the graph. This allows each node in the graph to access the store: when you define node functions, you can define `store` keyword argument, and LangGraph will automatically pass the store object you compiled the graph with.

When storing objects using the `Store` interface you define two things:

* the namespace for the object, a tuple (similar to directories)
* the object key (similar to filenames)

In our example, we'll be using `("memories", <user_id>)` as namespace and random UUID as key for each new memory.

Importantly, to determine the user, we will be passing `user_id` via the config keyword argument of the node function.

Let's first define a `RedisStore` already populated with some memories about the users.

In [3]:
from langchain_openai import OpenAIEmbeddings
from langgraph.store.redis import RedisStore
from langgraph.store.base import IndexConfig

# Set up Redis connection
REDIS_URI = "redis://redis:6379"

# Create index configuration for vector search
index_config: IndexConfig = {
    "dims": 1536,
    "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
    "ann_index_config": {
        "vector_type": "vector",
    },
    "distance_type": "cosine",
}

# Initialize the Redis store
redis_store = None
with RedisStore.from_conn_string(REDIS_URI, index=index_config) as s:
    s.setup()
    redis_store = s

20:42:20 langgraph.store.redis INFO   Redis standalone client detected for RedisStore.
20:42:20 redisvl.index.index INFO   Index already exists, not overwriting.
20:42:20 redisvl.index.index INFO   Index already exists, not overwriting.


## Create graph

In [4]:
import uuid

from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.redis import RedisSaver
from langgraph.store.base import BaseStore

model = ChatAnthropic(model="claude-3-5-sonnet-20241022")


# NOTE: we're passing the Store param to the node --
# this is the Store we compile the graph with
def call_model(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    namespace = ("memories", user_id)
    memories = store.search(namespace, query=str(state["messages"][-1].content))
    info = "\n".join([d.value["data"] for d in memories])
    system_msg = f"You are a helpful assistant talking to the user. User info: {info}"

    # Store new memories if the user asks the model to remember
    last_message = state["messages"][-1]
    if "remember" in last_message.content.lower():
        memory = "User name is Bob"
        store.put(namespace, str(uuid.uuid4()), {"data": memory})

    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")

# Set up Redis connection for checkpointer
REDIS_URI = "redis://redis:6379"
checkpointer = None
with RedisSaver.from_conn_string(REDIS_URI) as cp:
    cp.setup()
    checkpointer = cp

# NOTE: we're passing the store object here when compiling the graph
graph = builder.compile(checkpointer=checkpointer, store=redis_store)
# If you're using LangGraph Cloud or LangGraph Studio, you don't need to pass the store or checkpointer when compiling the graph, since it's done automatically.

20:42:20 langgraph.checkpoint.redis INFO   Redis client is a standalone client
20:42:20 redisvl.index.index INFO   Index already exists, not overwriting.
20:42:20 redisvl.index.index INFO   Index already exists, not overwriting.
20:42:20 redisvl.index.index INFO   Index already exists, not overwriting.


<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        If you're using LangGraph Platform or LangGraph Studio, you <strong>don't need</strong> to pass store when compiling the graph, since it's done automatically.
    </p>
</div>

## Run the graph!

Now let's specify a user ID in the config and tell the model our name:

In [5]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
input_message = {"role": "user", "content": "Hi! Remember: my name is Bob"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! Remember: my name is Bob
20:42:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
20:42:21 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
20:42:21 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 404 Not Found"


NotFoundError: Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-3-5-sonnet-20241022'}, 'request_id': 'req_011CVE7Pi9nafi1huJ3xWNfm'}

In [ ]:
config = {"configurable": {"thread_id": "2", "user_id": "1"}}
input_message = {"role": "user", "content": "what is my name?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

We can now inspect our Redis store and verify that we have in fact saved the memories for the user:

In [ ]:
for memory in redis_store.search(("memories", "1")):
    print(memory.value)

Let's now run the graph for another user to verify that the memories about the first user are self contained:

In [ ]:
config = {"configurable": {"thread_id": "3", "user_id": "2"}}
input_message = {"role": "user", "content": "what is my name?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()